In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
from numpy import asarray
from numpy import zeros
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Concatenate,Flatten,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
#import clean training data and test data

train_df = pd.read_csv('cleaned_data.csv')
test_df = pd.read_csv('test.csv')

In [3]:
# We still have some rows with nan value, so we have to remove those.

train_df[train_df.selected_text.isna()]

,textID,text,selected_text,sentiment
8698,12f21c8f19,star wars is boo i wanna do your job hand i...,NaN,positive
23992,699ccfe91f,trying to finish my ignmentplease god help me ...,NaN,negative


In [4]:
train_df.dropna(axis = 0, inplace=True)

In [5]:
#Finding start index of the selected_text in the text

def find_start_indices(x):
    text,sel_text  = x[0],x[1]
    text = text.split()
    sel_text = sel_text.split()
    end = sel_text[0]
    index = text.index(end)
    return index

#Finding end index of the selected_text in the text

def find_end_indices(x):
    text,sel_text,start_indices  = x[0],x[1],x[2]
    text = text.split()
    sel_text = sel_text.split()
    end = sel_text[-1]
    try:
        index = text.index(end,start_indices)
    except:
        index = text.index(end)
        
    return index

In [6]:
train_df['start_indices'] = train_df[['text','selected_text']].apply(lambda x:find_start_indices(x),axis=1)

train_df['end_indices'] = train_df[['text','selected_text','start_indices']].apply(lambda x:find_end_indices(x),axis=1)

In [7]:
train_df[train_df.end_indices<train_df.start_indices ].shape

# From here, we can see that there are few records where start index is greater than the end index. 
# So we have to remove those records from our training data.

(95, 6)

In [8]:
#Take only those rows into consideration where start index is smaller or equal to the end index.

train_df = train_df[train_df.end_indices>=train_df.start_indices ]

In [9]:
X = train_df[['text','sentiment']]
y = train_df[['start_indices','end_indices']]

In [10]:
X.shape,y.shape

((27279, 2), (27279, 2))

In [11]:
#Training and test data split 
x_train,x_val,y_train,y_val= train_test_split(X,y,test_size=0.20,random_state=42)


In [12]:
train_text = x_train['text'].values
val_text = x_val['text'].values

In [13]:
train_sentiment = x_train['sentiment'].values
val_sentiment = x_val['sentiment'].values

In [14]:
#FOR TEXT INPUT
tokenizer_text = Tokenizer(lower=True,split=' ',oov_token='oov')         #Tokenizer
tokenizer_text.fit_on_texts(train_text)
train_text=tokenizer_text.texts_to_sequences(train_text)
val_text=tokenizer_text.texts_to_sequences(val_text)
print(len(train_text),len(val_text))
vocab_size_text=len(tokenizer_text.word_index)+1
print(vocab_size_text)

max_length_text=40                                            #Let's take maximum length of text is 40 (40 words)
train_text = pad_sequences(train_text,maxlen=max_length_text,padding='post')
val_text = pad_sequences(val_text,maxlen=max_length_text,padding='post')
print(train_text.shape,val_text.shape)


#FOR SENTIMENT INPUT

tokenizer_sentiment = Tokenizer(lower=True,split=' ',oov_token='oov')
tokenizer_sentiment.fit_on_texts(train_sentiment)
train_sentiment=tokenizer_sentiment.texts_to_sequences(train_sentiment)
val_sentiment=tokenizer_sentiment.texts_to_sequences(val_sentiment)
print(len(train_sentiment),len(val_sentiment))
print(tokenizer_sentiment.word_index)
vocab_size_sentiment=len(tokenizer_sentiment.word_index)+1
print(vocab_size_sentiment)

max_length_sentiment=1                                           # We have only 1 word for the sentiment(positive, negative or neutral)
train_sentiment = pad_sequences(train_sentiment,maxlen=max_length_sentiment,padding='post')
val_sentiment = pad_sequences(val_sentiment,maxlen=max_length_sentiment,padding='post')
print(train_sentiment.shape,val_sentiment.shape)

21823 5456
22943
(21823, 40) (5456, 40)
21823 5456
{'oov': 1, 'neutral': 2, 'positive': 3, 'negative': 4}
5
(21823, 1) (5456, 1)


In [15]:
#Import glove vectors

embeddings_index = dict()
with open('glove.6B.300d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [16]:
#Get embeddings for all words

embedding_matrix = zeros((vocab_size_text, 300))
for word, i in tokenizer_text.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(22943, 300)


In [17]:
input1=Input(shape=(max_length_text,),name='input_text')
embed = Embedding(vocab_size_text,300,input_length=max_length_text,name='embedding',\
                      trainable=False,embeddings_initializer=tf.constant_initializer(embedding_matrix))(input1)
gru=GRU(32,name='GRU',return_sequences=True)(embed)
f1=Flatten()(gru)

input2=Input(shape=(max_length_sentiment,),name='input_sentiment')
embed2=Embedding(vocab_size_sentiment,10,input_length=max_length_sentiment,name='embedding_sentiment')(input2)
f2=Flatten()(embed2)


#Different layers of the model

concat1=Concatenate(axis=1)([f1,f2])
dense1=Dense(16,activation='relu',kernel_regularizer=l2(0.0001))(concat1)
drop1 = Dropout(0.6)(dense1)
ln= LayerNormalization()(drop1)
dense2=Dense(8,activation='relu',kernel_regularizer=l2(0.0001))(ln)
output=Dense(2,name='output')(dense2)

In [18]:
model=Model(inputs=[input1,input2],outputs=[output])                  #Instantiate model

In [19]:
model.summary()                             #Model summary

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 300)      6882900     input_text[0][0]                 
__________________________________________________________________________________________________
input_sentiment (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
GRU (GRU)                       (None, 40, 32)       32064       embedding[0][0]                  
______________________________________________________________________________________________

In [20]:
input_data = (train_text,train_sentiment)
output_data = y_train.values

val = (val_text,val_sentiment)
output_val = y_val.values
val_data = (val,output_val)

In [21]:
model.compile(optimizer='adam',loss="mse",metrics=["mae"])                #Compilation of model

In [25]:
#Fit model

model.fit(input_data,output_data,epochs=100,batch_size=128,validation_data=val_data,validation_batch_size=64)

Epoch 1/100
171/171 [==============================] - 6s 21ms/step - loss: 43.8555 - mae: 4.4391 - val_loss: 24.1119 - val_mae: 3.5770
Epoch 2/100
171/171 [==============================] - 3s 19ms/step - loss: 24.8402 - mae: 3.5573 - val_loss: 19.5305 - val_mae: 3.0589
Epoch 3/100
171/171 [==============================] - 3s 19ms/step - loss: 21.4571 - mae: 3.3254 - val_loss: 20.9225 - val_mae: 3.1209
Epoch 4/100
171/171 [==============================] - 3s 19ms/step - loss: 20.8529 - mae: 3.2677 - val_loss: 19.7503 - val_mae: 3.0444
Epoch 5/100
171/171 [==============================] - 3s 19ms/step - loss: 19.8459 - mae: 3.1557 - val_loss: 16.3768 - val_mae: 2.7946
Epoch 6/100
171/171 [==============================] - 3s 19ms/step - loss: 19.2734 - mae: 3.1359 - val_loss: 15.2234 - val_mae: 2.6209
Epoch 7/100
171/171 [==============================] - 3s 19ms/step - loss: 18.3544 - mae: 3.0478 - val_loss: 16.2133 - val_mae: 2.6446
Epoch 8/100
171/171 [===========================

Epoch 61/100
171/171 [==============================] - 3s 19ms/step - loss: 14.7314 - mae: 2.6353 - val_loss: 13.4424 - val_mae: 2.3119
Epoch 62/100
171/171 [==============================] - 3s 19ms/step - loss: 14.7676 - mae: 2.6790 - val_loss: 13.4343 - val_mae: 2.3177
Epoch 63/100
171/171 [==============================] - 3s 19ms/step - loss: 14.4864 - mae: 2.6204 - val_loss: 13.2377 - val_mae: 2.2451
Epoch 64/100
171/171 [==============================] - 3s 19ms/step - loss: 14.4557 - mae: 2.6292 - val_loss: 13.2929 - val_mae: 2.2627
Epoch 65/100
171/171 [==============================] - 3s 19ms/step - loss: 14.6970 - mae: 2.6565 - val_loss: 13.5778 - val_mae: 2.2762
Epoch 66/100
171/171 [==============================] - 3s 19ms/step - loss: 14.4656 - mae: 2.6203 - val_loss: 13.4605 - val_mae: 2.2804
Epoch 67/100
171/171 [==============================] - 3s 19ms/step - loss: 14.4830 - mae: 2.5947 - val_loss: 15.4828 - val_mae: 2.3643
Epoch 68/100
171/171 [===================

In [26]:
#Calculate mean squared value

pred_values = model.predict(input_data)
true_values = output_data

mse = tf.keras.losses.MeanSquaredError()
print(mse(true_values,pred_values))

tf.Tensor(10.826799, shape=(), dtype=float32)


In [28]:
pred = np.tile(pred,(true_values.shape[0],1))
print(pred)
print(pred.shape)

[[2.68143702 8.49145397]
 [2.68143702 8.49145397]
 [2.68143702 8.49145397]
 ...
 [2.68143702 8.49145397]
 [2.68143702 8.49145397]
 [2.68143702 8.49145397]]
(21823, 2)


In [29]:
print(mse(true_values,pred))

tf.Tensor(32.909339904785156, shape=(), dtype=float64)


In [30]:
print(train_df['start_indices'].value_counts()[:1])
print(train_df['end_indices'].value_counts()[:1])

pred = [0,3]
pred = np.tile(pred,(true_values.shape[0],1))
print(pred)
print(pred.shape)

0    16346
Name: start_indices, dtype: int64
3    2193
Name: end_indices, dtype: int64
[[0 3]
 [0 3]
 [0 3]
 ...
 [0 3]
 [0 3]
 [0 3]]
(21823, 2)


In [31]:
test_df

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [32]:
#To clean test data

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("\*+", "<CURSE>", text)
    return text

In [33]:
#Clean test data

test_df['text'] = test_df['text'].apply(lambda x: clean_text(x))

In [34]:
test_df

,textID,text,sentiment
0,f87dea47db,last session of the day,neutral
1,96d74cb729,shanghai is also really exciting precisely s...,positive
2,eee518ae67,recession hit veronique branquinho she has to ...,negative
3,01082688c6,happy bday,positive
4,33987a8ee5,i like it,positive
...,...,...,...
3529,e5f0e6ef4b,its at am im very tired but i cant sleep but...,negative
3530,416863ce47,all alone in this old house again thanks for ...,positive
3531,6332da480c,i know what you mean my little dog is sinking...,negative
3532,df1baec676,sutra what is your next youtube video gonna be...,positive


In [35]:
#Tokenize and padding for both inputs of test data

test_text = test_df['text'].values
test_text=tokenizer_text.texts_to_sequences(test_text)                           
test_text = pad_sequences(test_text,maxlen=max_length_text,padding='post')       
print(test_text.shape)

test_sentiment = test_df['sentiment'].values
test_sentiment=tokenizer_sentiment.texts_to_sequences(test_sentiment)
test_sentiment = pad_sequences(test_sentiment,maxlen=max_length_sentiment,padding='post')
print(test_sentiment.shape)


(3534, 40)
(3534, 1)


In [36]:
#Predict start index and end index for sentiment extraction.

results  = model.predict([test_text,test_sentiment])
results = np.round(results)
results

array([[ 0.,  2.],
       [ 1.,  3.],
       [ 9., 13.],
       ...,
       [ 9., 13.],
       [10., 14.],
       [ 0.,  2.]], dtype=float32)

In [37]:
test_df['start'],test_df['end'] = (abs(results[:,0])),(abs(results[:,1]))

In [38]:
#To find text between start index and end index

def find_sel_text(x):
    text = x[0]
    start = x[1]
    end = x[2]
    
    sel_text = text.split()[start:end+1]
    sel_text = " ".join(sel_text)
    return sel_text

In [39]:
test_df['start'] = test_df['start'].astype('int')
test_df['end'] = test_df['end'].astype('int')

In [40]:
test_df

,textID,text,sentiment,start,end
0,f87dea47db,last session of the day,neutral,0,2
1,96d74cb729,shanghai is also really exciting precisely s...,positive,1,3
2,eee518ae67,recession hit veronique branquinho she has to ...,negative,9,13
3,01082688c6,happy bday,positive,0,2
4,33987a8ee5,i like it,positive,0,3
...,...,...,...,...,...
3529,e5f0e6ef4b,its at am im very tired but i cant sleep but...,negative,1,4
3530,416863ce47,all alone in this old house again thanks for ...,positive,13,17
3531,6332da480c,i know what you mean my little dog is sinking...,negative,9,13
3532,df1baec676,sutra what is your next youtube video gonna be...,positive,10,14


In [41]:
test_df['selected_text'] = test_df[['text','start','end']].apply(lambda x:find_sel_text(x),axis=1)

In [42]:
test_df

,textID,text,sentiment,start,end,selected_text
0,f87dea47db,last session of the day,neutral,0,2,last session of
1,96d74cb729,shanghai is also really exciting precisely s...,positive,1,3,is also really
2,eee518ae67,recession hit veronique branquinho she has to ...,negative,9,13,company such a shame
3,01082688c6,happy bday,positive,0,2,happy bday
4,33987a8ee5,i like it,positive,0,3,i like it
...,...,...,...,...,...,...
3529,e5f0e6ef4b,its at am im very tired but i cant sleep but...,negative,1,4,at am im very
3530,416863ce47,all alone in this old house again thanks for ...,positive,13,17,me alive and kicking whoever
3531,6332da480c,i know what you mean my little dog is sinking...,negative,9,13,sinking into depression he wants
3532,df1baec676,sutra what is your next youtube video gonna be...,positive,10,14,i love your videos


In [ ]:
##NOW, TO CHECK THE MODEL WITH ANY RANDOM SENTENCE.

In [43]:
def get_sent(text, sentiment):
    test_df = pd.DataFrame(list(zip([text],[sentiment])), columns=['text','sentiment'])
    test_text = test_df['text'].values
    test_text=tokenizer_text.texts_to_sequences(test_text)
    test_text = pad_sequences(test_text,maxlen=max_length_text,padding='post')

    test_sentiment = test_df['sentiment'].values
    test_sentiment=tokenizer_sentiment.texts_to_sequences(test_sentiment)
    test_sentiment = pad_sequences(test_sentiment,maxlen=max_length_sentiment,padding='post')


    results  = model.predict([test_text,test_sentiment])
    results = np.round(results)

    test_df['start'],test_df['end'] = (abs(results[:,0])),(abs(results[:,1]))

    test_df['start'] = test_df['start'].astype('int')
    test_df['end'] = test_df['end'].astype('int')

    test_df['selected_text'] = test_df[['text','start','end']].apply(lambda x:find_sel_text(x),axis=1)
    return test_df['selected_text']

In [44]:
z = "I feel satisfied after having my lunch"

get_sent(z, "positive")

0    feel satisfied
Name: selected_text, dtype: object

In [45]:
z = "Today, I got yelled by my boss as he was very angry"

get_sent(z, "negative")

0    as he was very angry
Name: selected_text, dtype: object

In [46]:
z = "She manage her work very properly"

get_sent(z, "positive")

0    manage her work
Name: selected_text, dtype: object